# `AzureContentSafetyTextTool`

## Overview

### Integration details

>The `AzureContentSafetyTextTool` acts as a wrapper around the Azure AI Content Safety Service/API.
>The Tool will detect harmful content according to Azure's Content Safety Policy.

### Tool features

This integration allows for the detection of harmful or offensive content in text using Azure's Content Safety API. It supports four categories of harmful content: Sexual, Harm, Self-Harm, and Violence.

## Setup

This section provides details about how the Azure AI Content Safety integration works, including setup.

In [1]:
import getpass
import os

Now we can use the `AzureContentSafetyTextTool` combined with a model, using `create_react_agent`.

In [3]:
from langgraph.prebuilt import create_react_agent

In [7]:
from langchain_openai import AzureChatOpenAI

from libs.community.langchain_community.tools.azure_ai_services.content_safety import (
    AzureContentSafetyTextTool,
)

### Credentials

Credentials can be set by being passed as parameters and should be stored locally as environment variables.

In [14]:
os.environ["CONTENT_SAFETY_ENDPOINT"] = getpass.getpass()
os.environ["CONTENT_SAFETY_KEY"] = getpass.getpass()

In [4]:
content_endpoint = os.environ["CONTENT_SAFETY_ENDPOINT"]
content_key = os.environ["CONTENT_SAFETY_API_KEY"]

## Instantiation

Credentials can be passed directly, but they can also be retrieved automatically by the constructor if environment variables named `CONTENT_SAFETY_ENDPOINT` and `CONTENT_SAFETY_KEY` are set.

In [ ]:
os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass()
os.environ["OPENAI_API_VERSION"] = getpass.getpass()
os.environ["GPT_MODEL"] = getpass.getpass()
os.environ["AZURE_OPENAI_ENDPOINT"] = getpass.getpass()

In [8]:
model = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_deployment=os.environ["GPT_MODEL"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
)

In [9]:
cs = AzureContentSafetyTextTool(
    content_safety_key=content_key,
    content_safety_endpoint=content_endpoint,
)

tools = [cs]

Create a react agent to invoke the tool. 

In [10]:
agent = create_react_agent(model, tools)

## Invocation

Input must be in the form of a string (`str`).

In [11]:
input = "I hate you"

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)

Firstly, the tool can be used by directly passing input as an argument. However, this is discouraged as the tool is intended to be used in an executor chain.

In [ ]:
cs.invoke({"query": input})

### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

By using `.invoke`, the model can be told what to do and assess if using the tools it was given would assist in it's response. This is the intended use.

In [ ]:
agent.invoke(
    {
        "messages": [
            ("user", f"Can you check the following text for harmful content : {input}")
        ]
    }
)

## API reference

[Azure AI Content Safety Overview](https://learn.microsoft.com/azure/ai-services/content-safety/overview) | [Azure AI Content Safety Python API](https://learn.microsoft.com/python/api/overview/azure/ai-contentsafety-readme?view=azure-python)